# Dropout Prediction Notebook

In [1]:
import pandas as pd
students=pd.read_csv('students.csv')
sessions=pd.read_csv('user_sessions.csv')

In [7]:
sessions['login_time']=pd.to_datetime(sessions['login_time'],errors='coerce')
cutoff=sessions['login_time'].max()-pd.Timedelta(days=30)
# recent=sessions[sessions['login_time']>=cutoff]
# recent['activity_score']=recent[['pages_viewed','quizzes_attempted','duration_minutes']].sum(axis=1)
# activity=recent.groupby('student_id')['activity_score'].sum().reset_index()
# Filter recent sessions safely
recent = sessions[sessions['login_time'] >= cutoff].copy()

# Compute activity score without warnings
recent['activity_score'] = (
    recent[['pages_viewed', 'quizzes_attempted', 'duration_minutes']].sum(axis=1)
)


In [8]:
recent

,browser,created_at,device_os,device_type,duration_minutes,ip_hash,is_active,lessons_accessed,login_time,logout_time,pages_viewed,quizzes_attempted,region_code,session_id,session_token,student_id,activity_score
19,App,2026-01-04T07:08:00.000Z,Android,desktop,57,IP_98f13708,False,1,2026-01-04 07:08:00+00:00,2026-01-04T08:05:00.000Z,29,1,R01,20,tok_00000020_8882,2,87
24,Safari,2025-12-14T08:06:00.000Z,Android,desktop,93,IP_8e296a06,False,5,2025-12-14 08:06:00+00:00,2025-12-14T09:39:00.000Z,3,3,R03,25,tok_00000025_2209,2,99
35,Firefox,2025-12-19T06:27:00.000Z,Android,mobile,116,IP_19ca14e7,False,5,2025-12-19 06:27:00+00:00,2025-12-19T08:23:00.000Z,4,3,R05,36,tok_00000036_8850,3,123
42,Chrome,2026-01-04T06:40:00.000Z,Android,desktop,6,IP_17e62166,False,0,2026-01-04 06:40:00+00:00,2026-01-04T06:46:00.000Z,16,1,R02,43,tok_00000043_8134,3,23
50,Chrome,2025-12-07T16:34:00.000Z,Android,desktop,72,IP_2838023a,False,1,2025-12-07 16:34:00+00:00,2025-12-07T17:46:00.000Z,24,1,R03,51,tok_00000051_4750,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49976,App,2026-01-05T17:42:00.000Z,Windows,mobile,35,IP_8c6883dc,False,3,2026-01-05 17:42:00+00:00,2026-01-05T18:17:00.000Z,19,0,R05,18727,tok_00018727_9859,620,54
49982,Chrome,2025-12-17T15:06:00.000Z,iOS,desktop,5,IP_aabc3012,False,4,2025-12-17 15:06:00+00:00,2025-12-17T15:11:00.000Z,11,0,R05,18733,tok_00018733_9576,620,16
49987,Safari,2025-12-08T12:22:00.000Z,macOS,mobile,29,IP_52c64c72,False,2,2025-12-08 12:22:00+00:00,2025-12-08T12:51:00.000Z,20,1,R06,18738,tok_00018738_5557,620,50
49989,App,2025-12-17T19:58:00.000Z,Android,mobile,98,IP_705c03a1,False,5,2025-12-17 19:58:00+00:00,2025-12-17T21:36:00.000Z,28,2,R03,18740,tok_00018740_1910,620,128


In [9]:
master=students.merge(activity,on='student_id',how='left')
master['activity_score']=master['activity_score'].fillna(0)
th=master['activity_score'].median()
master['dropout_label']=(master['activity_score']<th).astype(int)
master.to_csv('prediction_master_dataset.csv',index=False)
master.head()

,age,created_at,current_level,date_of_birth,device_access,display_name,email,enrollment_date,grade,is_active,...,login_streak,onboarding_completed,preferred_language,school_id,section,student_code,student_id,total_points,activity_score,dropout_label
0,14,2023-09-20T00:00:00.000Z,12,2012-01-06,mobile_only,Student 1,student000001@learn.magicbus.edu,2023-09-20,9,True,...,41,False,Hindi,118,B,STU000001,1,2885,0.0,0
1,17,2023-05-20T00:00:00.000Z,1,2008-10-31,mobile_only,Student 2,student000002@learn.magicbus.edu,2023-05-20,11,True,...,13,True,Hindi,172,D,STU000002,2,52,186.0,0
2,17,2024-12-22T00:00:00.000Z,8,2008-10-02,shared_computer,Student 3,student000003@learn.magicbus.edu,2024-12-22,11,True,...,1,True,English,171,D,STU000003,3,1869,243.0,0
3,15,2025-02-19T00:00:00.000Z,4,2010-01-09,mobile_only,Student 4,student000004@learn.magicbus.edu,2025-02-19,9,True,...,33,True,English,179,B,STU000004,4,881,127.0,0
4,15,2024-07-14T00:00:00.000Z,10,2010-08-22,no_device,Student 5,student000005@learn.magicbus.edu,2024-07-14,9,True,...,41,True,English,104,B,STU000005,5,2314,76.0,0


In [11]:
import pandas as pd

# Read CSV file
csv_file = "students.csv"        # replace with your CSV file name
json_file = "students.json"     # desired JSON output file name

# Load CSV into DataFrame
df = pd.read_csv(csv_file)

# Convert to JSON and save
df.to_json(json_file, orient="records", indent=4)

print("JSON file created successfully:", json_file)

JSON file created successfully: students.json


In [15]:
import pandas as pd

# Read CSV file
csv_file = "user_sessions.csv"        # replace with your CSV file name
json_file = "user_sessions.json"     # desired JSON output file name

# Load CSV into DataFrame
df = pd.read_csv(csv_file)

# Convert to JSON and save
df.to_json(json_file, orient="records", indent=4)

print("JSON file created successfully:", json_file)

JSON file created successfully: user_sessions.json


In [16]:
students = pd.read_csv("students.csv")
sessions = pd.read_csv("user_sessions.csv")
progress = pd.read_csv("student_progress.csv")
skills = pd.read_csv("student_skills.csv")
achievements = pd.read_csv("student_achievements.csv")
quiz_responses = pd.read_csv("quiz_responses.csv")
career_interests = pd.read_csv("career_interests.csv")
pathway_skills = pd.read_csv("pathway_skills.csv")
skills_master = pd.read_csv("skills.csv")
skill_categories = pd.read_csv("skill_categories.csv")
quizzes = pd.read_csv("quizzes.csv")
career_pathways = pd.read_csv("career_pathways.csv")

In [27]:
rows = students.shape[0]      # number of rows
columns = students.shape[1]   # number of columns


print("Row count students:", rows)
print("Number of fields students:", columns)
students['student_id'].nunique()

Row count students: 15000
Number of fields students: 19


15000

In [39]:
import pandas as pd

# Load the CSV files
students = pd.read_csv("students.csv")


rows = students.shape[0]      # number of rows
columns = students.shape[1]   # number of columns

print("Row count sessions:", rows)
print("Number of fields sessions:", columns)

Row count sessions: 15000
Number of fields sessions: 19


In [51]:
import pandas as pd

# Load your dataset
df = pd.read_csv("student_progress.csv")

# Convert timestamps
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['updated_at'] = pd.to_datetime(df['updated_at'], errors='coerce')

# Convert TRUE/FALSE to boolean
df['bookmarked'] = df['bookmarked'].astype(str).str.upper().map({"TRUE": True, "FALSE": False})

# GROUP AND SUMMARIZE
student_summary = df.groupby('student_id').agg({

    # Engagement metrics
    'attempts_count': 'sum',
    'completion_percentage': 'mean',
    'time_spent_minutes': 'sum',

    # Counts of rows
    'lesson_id': 'count',

    # Bookmarks
    'bookmarked': 'sum',

    # Time-based metrics
    'created_at': 'min',
    'updated_at': 'max',

}).rename(columns={
    'lesson_id': 'lessons_attempted',
    'bookmarked': 'bookmarks_count',
    'attempts_count': 'total_attempts',
    'completion_percentage': 'avg_completion_percentage',
    'time_spent_minutes': 'total_time_spent',
    'created_at': 'first_activity',
    'updated_at': 'last_activity'
})

# Add completion, skipped, in-progress counts separately
status_counts = df.pivot_table(index='student_id', 
                               columns='status', 
                               aggfunc='size', 
                               fill_value=0)

# Merge with main summary
student_summary = student_summary.merge(status_counts, on='student_id', how='left')

# Device usage — most-used device
device_usage = (
    df.groupby('student_id')['device_used']
    .agg(lambda x: x.mode()[0])
    .rename('primary_device')
)

student_progress_s = student_summary.merge(device_usage, on='student_id', how='left')
student_progress_s = student_progress_s.reset_index()
# print(student_summary)
student_progress_s.head(5)
rows = student_progress_s.shape[0]      # number of rows
columns = student_progress_s.shape[1]   # number of columns

print("Row count sessions:", rows)
print("Number of fields sessions:", columns)
student_progress_s.columns

Row count sessions: 14985
Number of fields sessions: 13


Index(['student_id', 'total_attempts', 'avg_completion_percentage',
       'total_time_spent', 'lessons_attempted', 'bookmarks_count',
       'first_activity', 'last_activity', 'completed', 'in_progress',
       'not_started', 'skipped', 'primary_device'],
      dtype='object')

In [54]:
import pandas as pd

# Load data
df = pd.read_csv("user_sessions.csv")

# Convert time fields
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['login_time'] = pd.to_datetime(df['login_time'], errors='coerce')
df['logout_time'] = pd.to_datetime(df['logout_time'], errors='coerce')

# -------------------------------
# AGGREGATIONS
# -------------------------------

student_summary = df.groupby("student_id").agg({

    # Engagement metrics
    "session_id": "count",                          # total sessions
    "duration_minutes": ["sum", "mean", "max"],     # total / avg / longest session
    "pages_viewed": "sum",
    "quizzes_attempted": "sum",
    "lessons_accessed": "sum",

    # Time-based metrics
    "login_time": "min",      # first login
    "logout_time": "max",     # last login (most recent)
    
    # Device usage (mode)
    "browser": lambda x: x.mode()[0],
    "device_os": lambda x: x.mode()[0],
    "device_type": lambda x: x.mode()[0],

    # Region and IP
    "region_code": lambda x: x.mode()[0],
    "ip_hash": lambda x: x.nunique(),      # number of networks used

    # Active session signal
    "is_active": "mean"   # activity ratio
})

# Flatten MultiIndex columns
student_summary.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col 
                           for col in student_summary.columns]

# Rename to meaningful names
user_sessions_s = student_summary.rename(columns={
    "session_id_count": "total_sessions",
    "duration_minutes_sum": "total_time_spent",
    "duration_minutes_mean": "avg_session_duration",
    "duration_minutes_max": "longest_session",
    "pages_viewed_sum": "total_pages",
    "quizzes_attempted_sum": "total_quizzes",
    "lessons_accessed_sum": "total_lessons_accessed",
    "login_time_min": "first_login",
    "logout_time_max": "last_login",
    "ip_hash_<lambda>": "unique_ip_count",
    "is_active_mean": "active_session_ratio",
    "browser_<lambda>": "primary_browser",
    "device_os_<lambda>": "primary_os",
    "device_type_<lambda>": "primary_device",
    "region_code_<lambda>": "primary_region"
})
user_sessions_s = user_sessions_s.reset_index()
user_sessions_s.head(5)
# rows = user_sessions_s.shape[0]      # number of rows
# columns = user_sessions_s.shape[1]   # number of columns

# print("Row count sessions:", rows)
# print("Number of fields sessions:", columns)
user_sessions_s['student_id'].nunique()
# user_sessions_s.columns

1647

In [69]:
import pandas as pd

 # Join (merge) on student_id
merged_df = students.merge(user_sessions_s, on="student_id", how="inner")
# merged_df = merged_df.reset_index()
# print(merged_df.head())
# merged_df.count()

rows = merged_df.shape[0]      # number of rows
columns = merged_df.shape[1]   # number of columns

print("Row count sessions:", rows)
print("Number of fields sessions:", columns)
merged_df.head()
merged_df.columns

Row count sessions: 1647
Number of fields sessions: 34


Index(['age', 'created_at', 'current_level', 'date_of_birth', 'device_access',
       'display_name', 'email', 'enrollment_date', 'grade', 'is_active',
       'last_login_x', 'login_streak', 'onboarding_completed',
       'preferred_language', 'school_id', 'section', 'student_code',
       'student_id', 'total_points', 'total_sessions', 'total_time_spent',
       'avg_session_duration', 'longest_session', 'total_pages',
       'total_quizzes', 'total_lessons_accessed', 'first_login',
       'last_login_y', 'primary_browser', 'primary_os', 'primary_device',
       'primary_region', 'unique_ip_count', 'active_session_ratio'],
      dtype='object')

In [70]:
 # Join (merge) on student_id
merged_df = merged_df.merge(student_progress_s, on="student_id", how="inner")
# merged_df = merged_df.reset_index()
# print(merged_df.head())
# merged_df.count()

rows = merged_df.shape[0]      # number of rows
columns = merged_df.shape[1]   # number of columns

print("Row count sessions:", rows)
print("Number of fields sessions:", columns)

Row count sessions: 1647
Number of fields sessions: 46


In [72]:
merged_df.head()
# merged_df.columns

,age,created_at,current_level,date_of_birth,device_access,display_name,email,enrollment_date,grade,is_active,...,total_time_spent_y,lessons_attempted,bookmarks_count,first_activity,last_activity,completed,in_progress,not_started,skipped,primary_device_y
0,14,2023-09-20T00:00:00.000Z,12,2012-01-06,mobile_only,Student 1,student000001@learn.magicbus.edu,2023-09-20,9,True,...,42,3,0,2024-06-19 22:12:00+00:00,2024-09-23 06:33:00+00:00,0,0,1,2,desktop
1,17,2023-05-20T00:00:00.000Z,1,2008-10-31,mobile_only,Student 2,student000002@learn.magicbus.edu,2023-05-20,11,True,...,121,7,0,2024-06-21 06:53:00+00:00,2025-06-13 06:54:00+00:00,4,3,0,0,desktop
2,17,2024-12-22T00:00:00.000Z,8,2008-10-02,shared_computer,Student 3,student000003@learn.magicbus.edu,2024-12-22,11,True,...,130,6,0,2024-06-22 06:38:00+00:00,2025-11-08 16:40:00+00:00,4,1,1,0,tablet
3,15,2025-02-19T00:00:00.000Z,4,2010-01-09,mobile_only,Student 4,student000004@learn.magicbus.edu,2025-02-19,9,True,...,247,9,0,2024-06-05 14:14:00+00:00,2025-11-28 18:58:00+00:00,5,2,2,0,mobile
4,15,2024-07-14T00:00:00.000Z,10,2010-08-22,no_device,Student 5,student000005@learn.magicbus.edu,2024-07-14,9,True,...,270,7,2,2024-06-01 10:22:00+00:00,2025-08-05 06:09:00+00:00,6,0,1,0,desktop


In [73]:
merged_df.head(100).to_csv("output.csv")

In [82]:
import pandas as pd
import numpy as np

# -------------------------------------------------------
# LOAD + CLEAN
# -------------------------------------------------------
df = pd.read_csv("student_achievements.csv")

# normalize column names
df.columns = [c.strip().lower() for c in df.columns]

# Fix tz-aware timestamps → make tz-naive
for c in ["created_at", "earned_at"]:
    df[c] = pd.to_datetime(df[c], errors="coerce")
    df[c] = df[c].dt.tz_localize(None)  # <---- FIX APPLIED HERE

# Convert booleans
bool_cols = ["is_displayed", "notification_sent"]
for c in bool_cols:
    df[c] = df[c].astype(str).str.upper().map({"TRUE": True, "FALSE": False})

# Make all numeric fields numeric
num_cols = [
    "achievement_id", "points_awarded", "record_id",
    "shared_count", "source_id", "student_id", "times_earned"
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# -------------------------------------------------------
# FEATURE ENGINEERING
# -------------------------------------------------------
df["earned_date"]  = df["earned_at"].dt.date
df["earned_month"] = df["earned_at"].dt.to_period("M").astype(str)
df["earned_week"]  = df["earned_at"].dt.to_period("W").astype(str)
df["earned_year"]  = df["earned_at"].dt.year

# -------------------------------------------------------
# CATEGORY COUNTS
# -------------------------------------------------------
trigger_counts = (
    df.pivot_table(index="student_id",
                   columns="trigger_source",
                   values="achievement_id",
                   aggfunc="count",
                   fill_value=0)
    .add_prefix("count_")
)

# CATEGORY POINTS
trigger_points = (
    df.pivot_table(index="student_id",
                   columns="trigger_source",
                   values="points_awarded",
                   aggfunc="sum",
                   fill_value=0)
    .add_prefix("points_")
)

# -------------------------------------------------------
# STUDENT LEVEL AGGREGATION
# -------------------------------------------------------
student_summary = df.groupby("student_id").agg({

    # Activity volume
    "achievement_id": "count",
    "points_awarded": ["sum", "mean", "max"],

    # Engagement
    "times_earned": ["sum", "mean"],
    "shared_count": ["sum", "mean"],
    "source_id": pd.Series.nunique,

    # Display quality
    "is_displayed": "mean",
    "notification_sent": "mean",

    # Timeline
    "earned_at": ["min", "max"],
    "earned_date": pd.Series.nunique  # active days
})

# Flatten columns
student_summary.columns = [
    "_".join(col).strip() for col in student_summary.columns
]

# Rename columns to human-friendly
student_summary = student_summary.rename(columns={
    "achievement_id_count": "total_achievements",
    "points_awarded_sum": "total_points",
    "points_awarded_mean": "avg_points",
    "points_awarded_max": "max_points",
    "times_earned_sum": "total_times_earned",
    "times_earned_mean": "avg_times_earned",
    "shared_count_sum": "total_shared",
    "shared_count_mean": "avg_shared",
    "source_id_nunique": "unique_sources",
    "is_displayed_mean": "display_rate",
    "notification_sent_mean": "notification_rate",
    "earned_at_min": "first_achievement_at",
    "earned_at_max": "last_achievement_at",
    "earned_date_nunique": "active_days"
})

# -------------------------------------------------------
# DERIVED INSIGHTS
# -------------------------------------------------------
today = pd.Timestamp.today().normalize()

student_summary["days_since_last"] = (
    today - student_summary["last_achievement_at"]
).dt.days

student_summary["achievements_per_active_day"] = (
    student_summary["total_achievements"] /
    student_summary["active_days"].replace(0, np.nan)
)

student_summary["points_per_achievement"] = (
    student_summary["total_points"] /
    student_summary["total_achievements"].replace(0, np.nan)
)

# Consistency: average time gap between achievements
activity_window = (
    student_summary["last_achievement_at"] -
    student_summary["first_achievement_at"]
).dt.days

student_summary["avg_days_between_achievements"] = (
    activity_window /
    (student_summary["total_achievements"] - 1).replace(0, np.nan)
)

# -------------------------------------------------------
# FINAL MERGE
# -------------------------------------------------------
student_final = (
    student_summary
    .join(trigger_counts, how="left")
    .join(trigger_points, how="left")
    .reset_index()
)

# Optional: sort for presentation
student_achv_s = student_final.sort_values("total_points", ascending=False)

# print(student_final.head())
student_achv_s.head(5)
# print("Unique students:", student_final["student_id"].nunique())

,student_id,total_achievements,total_points,avg_points,max_points,total_times_earned,avg_times_earned,total_shared,avg_shared,unique_sources,...,points_per_achievement,avg_days_between_achievements,count_lesson_complete,count_module_complete,count_quiz_pass,count_streak_milestone,points_lesson_complete,points_module_complete,points_quiz_pass,points_streak_milestone
5539,5540,8,1475,184.375,500,8,1.0,19,2.375,8,...,184.375,63.428571,0,2,4,2,0,650,550,275
1094,1095,8,1365,170.625,500,8,1.0,24,3.000,8,...,170.625,59.142857,3,1,4,0,500,50,815,0
9202,9203,8,1350,168.750,500,8,1.0,22,2.750,8,...,168.750,56.714286,3,1,2,2,200,100,650,400
8102,8103,8,1350,168.750,500,8,1.0,15,1.875,8,...,168.750,58.571429,3,2,1,2,250,650,200,250
14151,14152,8,1345,168.125,500,8,1.0,16,2.000,8,...,168.125,74.142857,1,2,5,0,100,650,595,0


In [83]:
 # Join (merge) on student_id
merged_df = merged_df.merge(student_achv_s, on="student_id", how="inner")
# merged_df = merged_df.reset_index()
# print(merged_df.head())
# merged_df.count()

rows = merged_df.shape[0]      # number of rows
columns = merged_df.shape[1]   # number of columns

print("Row count sessions:", rows)
print("Number of fields sessions:", columns)

Row count sessions: 1647
Number of fields sessions: 72


In [84]:
merged_df.head(100).to_csv("output1.csv")